In [4]:
import numpy as np
import pandas as pd
import orca
import os; os.chdir('../')
import warnings; warnings.simplefilter('ignore')
from urbansim.utils import misc
import pandana as pdna
from collections import OrderedDict
    
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import pandas as pd
import orca
# import os; os.chdir('/home/juan/activitysynth/')
import warnings; warnings.simplefilter('ignore')
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
from sklearn.neighbors import BallTree

import pylogit as pl                   # For choice model estimation
from pylogit import nested_logit as nl # For nested logit convenience funcs
import math 
from collections import OrderedDict 

import pickle
import dill
import time
import random
import scipy.stats as st

import urbansim_templates

from scripts import datasources, models, variables, utils

In [6]:
input_file_format = 'csv'
input_data_dir = '/home/data/spring_2019/base/'

formattable_fname_dict = {
    'parcels': 'parcels.{0}',
    'buildings': 'buildings.{0}',
    'jobs': 'jobs.{0}',
    'establishments': 'establishments.{0}',
    'households': 'households.{0}',
    'persons': 'persons.{0}',
    'rentals': 'craigslist.{0}',
    'units': 'units.{0}',
    'mtc_skims': 'mtc_skims.{0}',
    'beam_skims_raw': '30.skims-smart-23April2019-baseline.csv.gz',
    'beam_skims_imputed': 'beam_skims_imputed.{0}',
    # the following nodes and edges .csv's will be phased out and
    # replaced by travel model skims entirely
    'drive_nodes': 'drive_nodes.{0}',
    'drive_edges': 'drive_edges.{0}',
    'drive_access_vars': 'drive_net_vars.{0}',
    'walk_nodes': 'walk_nodes.{0}',
    'walk_edges': 'walk_edges.{0}',
    'walk_access_vars': 'walk_net_vars.{0}',
    'zones': 'zones.{0}',
    'zone_access_vars': 'zones_w_access_vars.{0}',
}

def format_fname_dict(formattable_fname_dict, format='csv'):
    formatted_dict = {
        k: v.format('csv')
        for k, v in formattable_fname_dict.items()}
    return formatted_dict

input_fnames = format_fname_dict(
            formattable_fname_dict, input_file_format)

In [7]:
orca.add_injectable('input_file_format', input_file_format)
orca.add_injectable('input_data_dir', input_data_dir)
orca.add_injectable('input_fnames', input_fnames)
orca.add_injectable('store', None)

In [8]:
orca.run(['initialize_network_small','initialize_network_walk', 
          'initialize_imputed_skims','skims_aggregations'])

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Running step 'initialize_imputed_skims'
No imputed skims found. Creating them now.
Time to execute step 'initialize_imputed_skims': 95.57 s
Running step 'skims_aggregations'
Time to execute step 'skims_aggregations': 0.00 s
Total time to execute iteration 1 with iteration value None: 95.57 s


In [96]:
orca.run(['skims_aggregations'])

Running step 'skims_aggregations'
Time to execute step 'skims_aggregations': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s


In [97]:
orca.list_tables()

['parcels',
 'buildings',
 'jobs',
 'establishments',
 'households',
 'persons',
 'rentals',
 'units',
 'zones',
 'access_indicators_ampeak',
 'mtc_skims',
 'beam_skims_raw',
 'beam_skims_imputed',
 'drive_nodes',
 'drive_edges',
 'walk_nodes',
 'walk_edges',
 'schools',
 'public_schools_50',
 'private_schools_100',
 'students',
 'closest_schools',
 'long_format',
 'TOD_school_data_preparation']

In [98]:
persons_chts = pd.read_csv('/home/data/fall_2018/chts_persons_w_zone_ids.csv')
household_chts = pd.read_csv("/home/data/fall_2018/CHTS_csv_format/data/Deliv_HH.csv")
trips = pd.read_csv('/home/emma/activitysynth/activitysynth/data/HWtrips_042519.csv')
zones = orca.get_table('zones').to_frame()
beam_skims = orca.get_table('beam_skims_imputed').to_frame()

In [61]:
trips['SAMPN'] = [ str(x)[:-1] for x in trips.HHPER]
trips['SAMPN'] = trips['SAMPN'].astype(int)

In [68]:
trip = trips.loc[:,['SAMPN','PERNO','HHPER', 'origin','home_ST','home_dwell'
       ,'HW_trip_ST', 'HW_trip_ET', 'HW_TT','HW_mode',
       'work_ST', 'work_dwell', 'WH_trip_ST', 'WH_trip_ET', 'WH_TT', 'WH_mode',
                'Hzone','Wzone',]]

In [69]:
trip

,SAMPN,PERNO,HHPER,origin,home_ST,home_dwell,HW_trip_ST,HW_trip_ET,HW_TT,HW_mode,work_ST,work_dwell,WH_trip_ST,WH_trip_ET,WH_TT,WH_mode,Hzone,Wzone
0,3020219,3,30202193,home,7.166667,14.583333,21.750000,22.000000,0.250000,drive_alone,22.000000,9.000000,7.000000,7.166667,0.166667,drive_alone,525.0,523.0
1,2589341,1,25893411,home,17.783333,13.933333,7.716667,8.766667,1.050000,drive_alone,8.766667,8.033333,16.800000,17.783333,0.983333,drive_alone,183.0,69.0
2,2577141,1,25771411,home,18.483333,13.433333,7.916667,8.116667,0.200000,drive_alone,8.116667,10.116667,18.233333,18.483333,0.250000,drive_alone,358.0,345.0
3,3017740,1,30177401,home,18.316667,11.233333,5.550000,5.983333,0.433333,drive_alone,5.983333,8.683333,14.666667,18.316667,3.650000,drive_alone,524.0,362.0
4,1342753,1,13427531,home,17.666667,13.083333,6.750000,7.500000,0.750000,drive_alone,7.500000,8.500000,16.000000,17.666667,1.666667,drive_alone,1120.0,108.0
5,2988434,1,29884341,home,18.250000,14.416667,8.666667,9.583333,0.916667,DrTranW,9.583333,7.916667,17.500000,18.250000,0.750000,WTranDr,949.0,22.0
6,2232877,1,22328771,home,18.916667,15.133333,10.050000,10.350000,0.300000,drive_alone,10.350000,8.200000,18.550000,18.916667,0.366667,drive_alone,270.0,311.0
7,3023269,1,30232691,home,19.166667,13.916667,9.083333,9.500000,0.416667,drive_alone,9.500000,9.333333,18.833333,19.166667,0.333333,drive_alone,565.0,435.0
8,2495650,1,24956501,home,18.116667,12.133333,6.250000,7.416667,1.166667,shared,7.416667,9.666667,17.083333,18.116667,1.033333,shared,1242.0,90.0
9,3019997,1,30199971,home,11.750000,20.333333,8.083333,8.416667,0.333333,walk,8.416667,2.750000,11.166667,11.750000,0.583333,walk,517.0,516.0


In [73]:
trips_1 = trip.merge(household_chts, how = 'left', on = 'SAMPN')

In [84]:
not_include = ['WNAME_lookup', 'WCITY_lookup', 'WZIP_lookup',
       'WXST1_lookup', 'WXST2_lookup', 'WXCORD_lookup', 'WYCORD_lookup',
       'WNAME2_lookup', 'WCITY2_lookup', 'WZIP2_lookup', 'WXST2_1_lookup',
       'WXST2_2_lookup', 'SNAME_lookup', 'SCITY_lookup', 'SZIP_lookup',
       'SXST1_lookup', 'SXST2_lookup', 'SXCORD_lookup', 'SYCORD_lookup',
       'WCTFIP_lookup', 'WTRACT_lookup', 'WBLOCK', 'WPrimaryCity_lookup',
       'WSTFIP_lookup', 'W2PrimaryCity_lookup', 'W2STFIP_lookup',
       'SCTFIP_lookup', 'STRACT_lookup', 'SBLOCK', 'SPrimaryCity_lookup',
       'SSTFIP_lookup', 'HCITY', 'HZIP', 'HXCORD', 'HYCORD', 'HBLOCK',
       'HTRACT', 'HCTFIP', 'HPrimaryCity', 'parcel_id_home', 'parcel_id_work',
       'zone_id_home', 'zone_id_work', 'Unnamed: 0', 'index']
persons = persons_chts.drop(columns=not_include)

In [104]:
trips_2 = trips_1.merge(persons, how = 'left', on = ['SAMPN', 'PERNO'])

In [105]:
trips_2.shape

(3668, 243)

In [106]:
trips_3 = trips_2.merge(zones, how = 'left', left_on ='Hzone', right_index = True )
trips_3

,SAMPN,PERNO,HHPER,origin,home_ST,home_dwell,HW_trip_ST,HW_trip_ET,HW_TT,HW_mode,...,avg_income_gen_tt_WALK_TRANSIT_30,total_jobs_gen_tt_CAR_15,total_jobs_gen_tt_CAR_45,sum_persons_gen_tt_CAR_15,sum_persons_gen_tt_CAR_45,sum_income_gen_tt_CAR_15,sum_income_gen_tt_CAR_45,sum_residential_units_gen_tt_CAR_15,sum_residential_units_gen_tt_CAR_45,avg_income_gen_tt_CAR_30
0,3020219,3,30202193,home,7.166667,14.583333,21.750000,22.000000,0.250000,drive_alone,...,168307.818182,392669.0,860934.0,1062417.0,2190312.0,4.285544e+10,9.674394e+10,400820.0,848845.0,293032.689046
1,2589341,1,25893411,home,17.783333,13.933333,7.716667,8.766667,1.050000,drive_alone,...,184276.840912,344655.0,1071378.0,896188.0,2454933.0,4.532313e+10,1.162750e+11,412510.0,1127174.0,228182.599881
2,2577141,1,25771411,home,18.483333,13.433333,7.916667,8.116667,0.200000,drive_alone,...,224893.566759,260071.0,1236200.0,429889.0,3185993.0,2.570994e+10,1.345337e+11,189552.0,1253349.0,353524.148570
3,3017740,1,30177401,home,18.316667,11.233333,5.550000,5.983333,0.433333,drive_alone,...,161216.374536,356756.0,877703.0,1033579.0,2207182.0,4.213307e+10,9.742089e+10,392509.0,857993.0,286302.640523
4,1342753,1,13427531,home,17.666667,13.083333,6.750000,7.500000,0.750000,drive_alone,...,175228.361038,152353.0,975342.0,360407.0,2508678.0,1.576870e+10,1.010757e+11,152263.0,1100157.0,177725.813026
5,2988434,1,29884341,home,18.250000,14.416667,8.666667,9.583333,0.916667,DrTranW,...,140983.550873,279026.0,1538101.0,630098.0,3549524.0,2.283539e+10,1.595315e+11,294384.0,1563463.0,247561.802928
6,2232877,1,22328771,home,18.916667,15.133333,10.050000,10.350000,0.300000,drive_alone,...,300883.676953,163145.0,1618529.0,331336.0,3383490.0,1.871636e+10,1.593826e+11,147956.0,1478779.0,279151.745331
7,3023269,1,30232691,home,19.166667,13.916667,9.083333,9.500000,0.416667,drive_alone,...,156837.528673,384335.0,866673.0,998280.0,2122681.0,3.923950e+10,9.441426e+10,379891.0,827777.0,210101.950236
8,2495650,1,24956501,home,18.116667,12.133333,6.250000,7.416667,1.166667,shared,...,75135.856851,50026.0,576803.0,201381.0,1698842.0,6.094695e+09,6.664311e+10,83357.0,752911.0,162804.584493
9,3019997,1,30199971,home,11.750000,20.333333,8.083333,8.416667,0.333333,walk,...,134398.262361,260973.0,842528.0,694275.0,2057750.0,3.295054e+10,9.276874e+10,290593.0,807794.0,259016.280165


In [99]:
zones

,gid,area,acres,total_jobs,sum_residential_units,sum_persons,sum_income,avg_income,total_jobs_gen_tt_WALK_TRANSIT_15,total_jobs_gen_tt_WALK_TRANSIT_45,...,avg_income_gen_tt_WALK_TRANSIT_30,total_jobs_gen_tt_CAR_15,total_jobs_gen_tt_CAR_45,sum_persons_gen_tt_CAR_15,sum_persons_gen_tt_CAR_45,sum_income_gen_tt_CAR_15,sum_income_gen_tt_CAR_45,sum_residential_units_gen_tt_CAR_15,sum_residential_units_gen_tt_CAR_45,avg_income_gen_tt_CAR_30
zone_id,,,,,,,,,,,,,,,,,,,,,
1442,525,5.272042e+06,121.028999,2194.0,2567.0,5839.0,477004000.0,207557.460533,6535.0,15529.0,...,389859.575835,77852.0,817038.0,172198.0,1905753.0,1.061663e+10,9.200540e+10,82479.0,894586.0,324956.320230
1434,508,7.820342e+06,179.529999,771.0,2425.0,5237.0,313941043.0,141307.478037,2645.0,11639.0,...,278758.962238,32692.0,652413.0,84295.0,1681229.0,5.005057e+09,7.959563e+10,37851.0,791987.0,262874.802420
246,784,4.247868e+06,97.517998,4260.0,2946.0,7469.0,511362913.0,218191.372815,4260.0,4260.0,...,218191.372815,173884.0,1373518.0,360301.0,2984104.0,1.812744e+10,1.412099e+11,151778.0,1333556.0,347640.029933
1312,560,7.597325e+08,17441.058594,220.0,1018.0,1591.0,65422447.0,86841.113816,220.0,220.0,...,86841.113816,440.0,20452.0,3182.0,27961.0,1.308449e+08,1.370876e+09,2036.0,16441.0,189294.231209
705,1205,2.783539e+07,639.013000,948.0,3682.0,9915.0,372431913.0,117368.284928,1667.0,2497.0,...,214184.516958,19076.0,134984.0,76673.0,635582.0,2.694645e+09,2.362599e+10,26407.0,221699.0,237441.486169
1291,552,7.535315e+07,1729.869995,818.0,3519.0,9542.0,307786228.0,102837.862813,818.0,818.0,...,102837.862813,51785.0,509755.0,194353.0,1485553.0,6.062375e+09,5.710460e+10,79693.0,653029.0,184662.743958
875,1448,6.510720e+06,149.466003,11746.0,1071.0,3317.0,57754116.0,57780.755751,11746.0,27258.0,...,57780.755751,232955.0,1573302.0,639728.0,3515717.0,2.063902e+10,1.593641e+11,279430.0,1547645.0,163496.101093
1087,435,1.121201e+08,2573.924072,208.0,2464.0,5640.0,281186242.0,138147.468964,208.0,208.0,...,138147.468964,67190.0,910799.0,243285.0,2461845.0,8.421288e+09,1.042382e+11,100729.0,1094613.0,240372.168657
1316,561,1.817411e+08,4172.201172,999.0,822.0,1112.0,55971098.0,100597.241807,999.0,999.0,...,100597.241807,8387.0,156683.0,11540.0,431777.0,5.108199e+08,1.533341e+10,6723.0,199108.0,189160.611760


In [111]:
beam_skims = beam_skims.reset_index()
beam_skims

,index,from_zone_id,to_zone_id,dist,gen_cost_BIKE,gen_cost_CAR,gen_cost_DRIVE_TRANSIT,gen_cost_RIDE_HAIL,gen_cost_RIDE_HAIL_POOLED,gen_cost_RIDE_HAIL_TRANSIT,gen_cost_WALK,gen_cost_WALK_TRANSIT,gen_tt_BIKE,gen_tt_CAR,gen_tt_DRIVE_TRANSIT,gen_tt_RIDE_HAIL,gen_tt_RIDE_HAIL_POOLED,gen_tt_RIDE_HAIL_TRANSIT,gen_tt_WALK,gen_tt_WALK_TRANSIT
0,0,1,1,193.1208,0.248434,0.114126,0.145693,0.576147,0.316090,0.309961,28.510923,3.505719,0.805269,0.173001,0.286490,0.310197,0.241026,0.273580,3.395910,1.850000
1,1,1,2,386.2416,0.496869,0.228251,0.291386,1.152294,0.632179,0.619923,57.021846,22.054913,1.610537,0.346003,0.572980,0.620393,0.482052,0.547160,6.791820,23.037028
2,2,1,3,708.1096,0.910926,0.418461,0.534207,2.112539,1.158995,1.136525,104.540052,40.434006,2.952651,0.634338,1.050464,1.137388,0.883762,1.003127,12.451669,42.234551
3,3,1,4,659.8294,0.848817,0.389929,0.497784,1.968503,1.079973,1.059035,97.412321,37.677142,2.751334,0.591088,0.978841,1.059839,0.823505,0.934732,11.602692,39.354922
4,4,1,5,1094.3512,1.407795,0.646712,0.825593,3.264834,1.791175,1.756448,161.561898,62.488919,4.563188,0.980341,1.623444,1.757781,1.365814,1.550287,19.243489,65.271579
5,5,1,6,1561.0598,2.008178,0.922515,1.177684,4.657189,2.555058,2.505522,230.463296,89.138605,6.509254,1.398427,2.315795,2.507423,1.948293,2.211438,27.450271,93.107987
6,6,1,7,1577.1532,2.028881,0.932026,1.189825,4.705201,2.581399,2.531352,232.839206,90.057560,6.576360,1.412844,2.339669,2.533273,1.968379,2.234237,27.733264,94.067863
7,7,1,8,1850.7410,2.380829,1.093704,1.396223,5.521410,3.029193,2.970464,273.229681,105.679790,7.717157,1.657929,2.745530,2.972718,2.309832,2.621808,32.544136,110.385758
8,8,1,9,2526.6638,3.250350,1.493143,1.906148,7.537925,4.135506,4.055330,373.017912,144.275887,10.535597,2.263434,3.748246,4.058406,3.153423,3.579338,44.429821,150.700557
9,9,1,10,2558.8506,3.291756,1.512164,1.930431,7.633949,4.188188,4.106990,377.769732,146.113796,10.669808,2.292268,3.795994,4.110106,3.193594,3.624935,44.995805,152.620309


In [112]:
trips_4 = trips_3.merge(beam_skims, 
                        how = 'left', 
                        left_on = ['Hzone','Wzone'], 
                        right_on = ['from_zone_id','to_zone_id'])

In [113]:
trips_4

,SAMPN,PERNO,HHPER,origin,home_ST,home_dwell,HW_trip_ST,HW_trip_ET,HW_TT,HW_mode,...,gen_cost_WALK,gen_cost_WALK_TRANSIT,gen_tt_BIKE,gen_tt_CAR,gen_tt_DRIVE_TRANSIT,gen_tt_RIDE_HAIL,gen_tt_RIDE_HAIL_POOLED,gen_tt_RIDE_HAIL_TRANSIT,gen_tt_WALK,gen_tt_WALK_TRANSIT
0,3020219,3,30202193,home,7.166667,14.583333,21.750000,22.000000,0.250000,drive_alone,...,472.806143,182.871984,13.354037,2.868939,4.750961,8.633333,3.997014,4.536868,56.315505,191.015355
1,2589341,1,25893411,home,17.783333,13.933333,7.716667,8.766667,1.050000,drive_alone,...,1278.239723,494.397624,36.102874,10.333333,12.844308,13.907150,10.805997,12.265502,152.249958,516.413372
2,2577141,1,25771411,home,18.483333,13.433333,7.916667,8.116667,0.200000,drive_alone,...,1185.579223,458.558391,33.485751,7.250000,11.913215,12.899011,10.022662,11.376368,141.213251,478.978202
3,3017740,1,30177401,home,18.316667,11.233333,5.550000,5.983333,0.433333,drive_alone,...,4785.083277,1850.774748,135.150906,26.183333,48.082594,52.061340,40.452188,45.915840,569.946870,1933.190579
4,1342753,1,13427531,home,17.666667,13.083333,6.750000,7.500000,0.750000,drive_alone,...,7096.843966,2744.917663,200.444765,80.350000,71.312169,77.213119,59.995376,68.098617,845.298560,2867.150079
5,2988434,1,29884341,home,18.250000,14.416667,8.666667,9.583333,0.916667,DrTranW,...,3464.077169,1339.835940,97.840130,21.019661,24.977778,37.688894,29.284653,33.239968,412.603047,1399.499436
6,2232877,1,22328771,home,18.916667,15.133333,10.050000,10.350000,0.300000,drive_alone,...,9.151144,473.261666,34.559442,2.850000,12.295202,2.070833,10.344030,11.741141,34.675000,494.336221
7,3023269,1,30232691,home,19.166667,13.916667,9.083333,9.500000,0.416667,drive_alone,...,2000.516445,9.039379,56.503010,10.489583,20.102058,21.765466,22.190000,19.196195,238.279674,42.866667
8,2495650,1,24956501,home,18.116667,12.133333,6.250000,7.416667,1.166667,shared,...,11299.829228,4370.548512,319.154772,68.566193,113.545589,122.941277,95.526618,108.428866,1345.912270,4565.171000
9,3019997,1,30199971,home,11.750000,20.333333,8.083333,8.416667,0.333333,walk,...,468.054323,181.034074,13.219825,2.840105,4.703213,2.900000,3.956843,4.491271,55.749520,189.095603


In [115]:
os.getcwd()

'/home/juan'

In [116]:
trips_4.to_csv('/home/juan/work_trips.csv')